# DCGAN_MNIST

- Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks(https://arxiv.org/pdf/1511.06434.pdf)

![alt text](./dcgan.png)

## 1. Import Libs 

In [1]:
import os
import matplotlib.pyplot as plt
import itertools
import imageio
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
from collections import OrderedDict
import time

## 2. Hyperparameters

In [2]:
num_epochs = 20
# num_epochs = 2
learning_rate = 0.0002
batch_size = 128

## 3. Data

In [3]:
# data_loader
img_size = 64

mnist_train = dset.MNIST(
    root='./data',
    train=True, 
    download=True, 
    transform=transforms.Compose([
        transforms.Scale(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ])
)

In [4]:
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)

## 4. Model
### 1) Generator

In [5]:
class Generator(nn.Module):
    def __init__(self, dim=128):
        super(Generator, self).__init__()
        self.deconv1 = nn.ConvTranspose2d(100, dim*8, 4, 1, 0)
        self.bn1 =  nn.BatchNorm2d(dim*8)
        self.deconv2 = nn.ConvTranspose2d(dim*8, dim*4, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(dim*4)
        self.deconv3 = nn.ConvTranspose2d(dim*4, dim*2, 4, 2 ,1)
        self.bn3 = nn.BatchNorm2d(dim*2)
        self.deconv4 = nn.ConvTranspose2d(dim*2, dim, 4, 2, 1)
        self.bn4 = nn.BatchNorm2d(dim)
        self.deconv5 = nn.ConvTranspose2d(dim, 1, 4, 2, 1)

    # 가중치 초기화 설정 함수
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)
    
    def forward(self, z):
        z = F.relu(self.bn1(self.deconv1(z)))
        z = F.relu(self.bn2(self.deconv2(z)))
        z = F.relu(self.bn3(self.deconv3(z)))
        z = F.relu(self.bn4(self.deconv4(z)))
        z = F.tanh(self.deconv5(z))
        return z

### 2) Discriminator

In [6]:
class Discriminator(nn.Module):
    def __init__(self, dim=128):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(1, dim, 4, 2, 1)
        self.conv2 = nn.Conv2d(dim, dim*2, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(dim*2)
        self.conv3 = nn.Conv2d(dim*2, dim*4, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(dim*4)
        self.conv4 = nn.Conv2d(dim*4, dim*8, 4, 2, 1)
        self.bn4 = nn.BatchNorm2d(dim*8)
        self.conv5 = nn.Conv2d(dim*8, 1, 4, 1, 0)
        
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)
            
    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = F.leaky_relu(self.bn2(self.conv2(x)), 0.2)
        x = F.leaky_relu(self.bn3(self.conv3(x)), 0.2)
        x = F.leaky_relu(self.bn4(self.conv4(x)), 0.2)
        x = F.sigmoid(self.conv5(x))
        return x
    
def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

## 5. Multi-gpu

In [7]:
generator = Generator(64)
discriminator = Discriminator(64)
generator.weight_init(mean = 0.0, std = 0.02)
discriminator.weight_init(mean=0.0, std = 0.02)

generator = nn.DataParallel(Generator()).cuda()
discriminator = nn.DataParallel(Discriminator()).cuda()

In [8]:
gen_params = generator.state_dict().keys()
dis_params = discriminator.state_dict().keys()
for param in gen_params:
    print(param)

module.deconv1.weight
module.deconv1.bias
module.bn1.weight
module.bn1.bias
module.bn1.running_mean
module.bn1.running_var
module.deconv2.weight
module.deconv2.bias
module.bn2.weight
module.bn2.bias
module.bn2.running_mean
module.bn2.running_var
module.deconv3.weight
module.deconv3.bias
module.bn3.weight
module.bn3.bias
module.bn3.running_mean
module.bn3.running_var
module.deconv4.weight
module.deconv4.bias
module.bn4.weight
module.bn4.bias
module.bn4.running_mean
module.bn4.running_var
module.deconv5.weight
module.deconv5.bias


## 6. Optimizer & Loss func

In [9]:
# Binary Cross Entropy loss
BCE_loss = nn.BCELoss()

# Adam
gen_optim = optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
dis_optim = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))

## 7. etc

In [10]:
if not os.path.isdir('MNIST_DCGAN_results'):
    os.mkdir('MNIST_DCGAN_results')
if not os.path.isdir('MNIST_DCGAN_results/Random_results'):
    os.mkdir('MNIST_DCGAN_results/Random_results')
if not os.path.isdir('MNIST_DCGAN_results/Fixed_results'):
    os.mkdir('MNIST_DCGAN_results/Fixed_results')

In [11]:
fixed_z_ = torch.randn((5 * 5, 100)).view(-1, 100, 1, 1)    # fixed noise
fixed_z_ = Variable(fixed_z_.cuda(), volatile=True)
#fixed_z_ = Variable(fixed_z_, volatile=True)

In [12]:
def show_result(num_epoch, show = False, save = False, path = 'result.png', isFix=False):
    z_ = torch.randn((5*5, 100)).view(-1, 100, 1, 1)
    z_ = Variable(z_.cuda(), volatile=True)
    #z_ = Variable(z_, volatile=True)

    generator.eval()
    if isFix:
        test_images = generator(fixed_z_)
    else:
        test_images = generator(z_)
    generator.train()

    size_figure_grid = 5
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(5, 5))
    for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)

    for k in range(5*5):
        i = k // 5
        j = k % 5
        ax[i, j].cla()
        ax[i, j].imshow(test_images[k, 0].cpu().data.numpy(), cmap='gray')

    label = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0.04, label, ha='center')
    plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

In [13]:
def show_train_hist(hist, show = False, save = False, path = 'Train_hist.png'):
    x = range(len(hist['discriminator_losses']))

    y1 = hist['discriminator_losses']
    y2 = hist['generator_losses']

    plt.plot(x, y1, label='discriminator_loss')
    plt.plot(x, y2, label='generator_losses')

    plt.xlabel('Iteration')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

## 8. Train

In [14]:
train_hist = {}
train_hist['discriminator_losses'] = []
train_hist['generator_losses'] = []
train_hist['per_epoch_ptimes'] = []
train_hist['total_ptime'] = []
num_iter = 0

start_time = time.time()
for epoch in range(num_epochs):
    dis_losses = []
    gen_losses = []
    epoch_start_time = time.time()
    # print("epoch : ", epoch)
    ## 1 epoch -> 10분 ===> 200분 : 3시간~4시간반
    for step, [image, _] in enumerate(train_loader): # step 0 ~ 468 :
        
        ##### Train Discriminator
        discriminator.zero_grad()
        
        '''
        print("train_loader's step : ", step)
        print("==========before Variable Image=======")
        print(image) # [torch.FloatTensor of size 128x1x64x64]
        '''
        mini_batch = image.size()[0] # 128
        
        y_real = torch.ones(mini_batch) # [torch.FloatTensor of size 128]
        y_fake = torch.zeros(mini_batch)
        
        #image = Variable(image)
        #y_real = Variable(y_real) # discriminator와 generator모두에서 사용
        #y_fake = Variable(y_fake)  # discriminator에서만 사용
       
        image = Variable(image.cuda())
        y_real = Variable(y_real.cuda()) # discriminator와 generator모두에서 사용
        y_fake = Variable(y_fake.cuda())  # discriminator에서만 사용
       
  
        '''
        print("==========after Variable Image=======")
        print(image) # [torch.FloatTensor of size 128x1x64x64]
        '''
        dis_result = discriminator(image).squeeze()
        '''
        print("==========after Variable Image=======")
        print("==========and Squeeze!!!!!!!=======")
        print(dis_result) # [torch.FloatTensor of size 128]
        '''
        dis_real_loss = BCE_loss(dis_result, y_real)
        
        # [torch.FloatTensor of size 128x100x1x1]
        z_ = torch.randn((mini_batch, 100)).view(-1, 100, 1, 1)
        #z_ = Variable(z_)
        z_ = Variable(z_.cuda())
        gen_result = generator(z_) # [torch.FloatTensor of size 128x1x64x64]
        
        '''
        dis_result = discriminator(gen_result)
        print(dis_result) # 128 x 1 x 1 x 1
        dis_result2 = discriminator(gen_result).squeeze()
        print(dis_result2) # [torch.FloatTensor of size 128]
        '''
        
        dis_result = discriminator(gen_result).squeeze()
        dis_fake_loss = BCE_loss(dis_result, y_fake)
        dis_fake_score = dis_result.data.mean()
        
        dis_train_loss = dis_real_loss + dis_fake_loss
        
        dis_train_loss.backward()
        dis_optim.step()
        
        dis_losses.append(dis_train_loss.data[0])
        
        '''
        print("dis_fake_loss : ", dis_fake_loss)
        print("dis_real_loss", dis_real_loss)
        print("dis_fake_score", dis_fake_score)
        '''
        
        ##### Train Generator
        generator.zero_grad()
        
        z_ = torch.randn((mini_batch, 100)).view(-1, 100, 1, 1)
        #z_ = Variable(z_)
        z_ = Variable(z_.cuda())
        
        
        gen_result = generator(z_)
        dis_result = discriminator(gen_result).squeeze()
        gen_train_loss = BCE_loss(dis_result, y_real)
        gen_train_loss.backward()
        gen_optim.step()
        
        gen_losses.append(gen_train_loss.data[0])
        
        num_iter += 1
        print(step)
    epoch_end_time = time.time()
    per_epoch_ptime = epoch_end_time - epoch_start_time
    print('[%d/%d] - ptime: %.2f, loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), 
                                                                 num_epochs, per_epoch_ptime, 
                                                                 torch.mean(torch.FloatTensor(dis_losses)), 
                                                                 torch.mean(torch.FloatTensor(gen_losses))))
    p = 'MNIST_DCGAN_results/Random_results/MNIST_DCGAN_' + str(epoch + 1) + '.png'
    fixed_p = 'MNIST_DCGAN_results/Fixed_results/MNIST_DCGAN_' + str(epoch + 1) + '.png'
    show_result((epoch+1), save=True, path=p, isFix=False)
    show_result((epoch+1), save=True, path=fixed_p, isFix=True)
    train_hist['discriminator_losses'].append(torch.mean(torch.FloatTensor(dis_losses)))
    train_hist['generator_losses'].append(torch.mean(torch.FloatTensor(gen_losses)))

end_time = time.time()
total_ptime = end_time - start_time
train_hist['total_ptime'].append(total_ptime)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
[5/20] - ptime: 549.43, loss_d: 0.598, loss_g: 2.952
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
3

8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
28

260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
[14/20] - ptime: 538.82, loss_d: 0.491, loss_g: 3.419
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
2

## 9. Save Model & visualize

In [ ]:
#print("Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f" % (torch.mean(torch.FloatTensor(train_hist['per_epoch_ptimes'])), num_epochs, total_ptime))
#print("Training finish!... save training results")

In [17]:
import pickle
torch.save(generator.state_dict(), "MNIST_DCGAN_results/generator_param.pkl")
torch.save(discriminator.state_dict(), "MNIST_DCGAN_results/discriminator_param.pkl")
with open('MNIST_DCGAN_results/train_hist.pkl', 'wb') as f:
    pickle.dump(train_hist, f)
    
show_train_hist(train_hist, save=True, path='MNIST_DCGAN_results/MNIST_DCGAN_train_hist.png')

## 10. img2gif

In [18]:
images = []
for e in range(num_epochs):
    img_name = 'MNIST_DCGAN_results/Fixed_results/MNIST_DCGAN_' + str(e + 1) + '.png'
    images.append(imageio.imread(img_name))
imageio.mimsave('MNIST_DCGAN_results/generation_animation.gif', images, fps=5)